In [ ]:
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import Dense
import pandas as pd
from math import ceil,floor

import scipy
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import StratifiedShuffleSplit

from time import time
from tensorflow.keras.callbacks import TensorBoard

from plotly import tools
import plotly as py
import plotly.graph_objs as g_obj
py.tools.set_credentials_file(username='', api_key='')

def custom_plot3d(x_, y_, z_, Y): 
    
    #https://plot.ly/python/3d-subplots/
    fig_obj3d = g_obj.Scatter3d(x=x_, y=y_, z=z_, mode='markers', \
                        marker=dict( size=3, color=Y, colorscale='Rainbow', opacity=0.8))
    fig_objXY = g_obj.Scatter(x=x_, y=y_, mode='markers', \
                        marker=dict( size=3, color=Y, colorscale='Rainbow', opacity=0.8))
    fig_objXZ = g_obj.Scatter(x=x_, y=z_, mode='markers', \
                        marker=dict( size=3, color=Y, colorscale='Rainbow', opacity=0.8))
    fig_objYZ = g_obj.Scatter(x=y_, y=z_, mode='markers', \
                        marker=dict( size=3, color=Y, colorscale='Rainbow', opacity=0.8))
    
    fig = tools.make_subplots(rows=2, cols=2, specs=[[{'is_3d': True}, {'is_3d': False}],\
                                                     [{'is_3d': False}, {'is_3d': False}]])
    
    fig.append_trace(fig_obj3d, 1, 1)
    fig.append_trace(fig_objXY, 1, 2)
    fig.append_trace(fig_objXZ, 2, 1)
    fig.append_trace(fig_objYZ, 2, 2)
    
    fig['layout'].update(height=1200, width=1000)
    
    return py.plotly.iplot(fig, filename='Dataset:')


def interquartile_range(X, Y, i):
    X_i = X[Y==int(i),:]
    sorted_x = np.sort(X_i, axis=0)
    n = np.shape(sorted_x)[0]
    
    if (n%2 != 0):
        Q1 = (sorted_x[ceil(n*0.25),:]+sorted_x[floor(n*0.25),:])/2
        Q3 = (sorted_x[ceil(n*0.75),:]+sorted_x[floor(n*0.75),:])/2
    else:
        Q1 = sorted_x[ceil(n*0.25),:]
        Q3 = sorted_x[ceil(n*0.75),:]
    
    coeff = [2.5, 2.5, 2.5]
    iqr = Q3-Q1
    cut_off = np.absolute(iqr*coeff)
    lower = Q1 - cut_off
    upper = Q3 + cut_off
    
    x = np.logical_and(X_i[:,0] < upper[0], X_i[:,0] > lower[0]) 
    y = np.logical_and(X_i[:,1] < upper[1], X_i[:,1] > lower[1])
    z = np.logical_and(X_i[:,2] < upper[2], X_i[:,2] > lower[2])
    
    to_mantain = x & y & z
    to_cut = np.logical_not(to_mantain)
    
    X_in = X_i[to_mantain, :]
    X_out = X_i[to_cut, :]
    
    return X_in, X_out 



'''
LOAD DATA
'''
dataset = pd.read_csv('LabeledMarkers_formatted.csv', sep=';',decimal=",", names=['Class', 'x', 'y', 'z'], \
                        skiprows=1, dtype={'Class':int, 'x':float, 'y':float, 'z':float})
X_loaded = dataset.iloc[:, 1:4].values
Y_loaded = dataset.iloc[:,0].values


'''
    PLOTTING ALL DATASET
'''
#custom_plot3d(X[:,0], X[:,1], X[:,2],Y)


In [ ]:
'''
    PREPROCESING: OUTLIERS & NORMALIZATION
'''
#INTERQUARTILE RANGE
X = np.empty([0,3])
Y = np.empty([0,1], dtype=int)
data = []
for i in range(1,12):
    X_in, X_out = interquartile_range(X_loaded, Y_loaded, i)
    X = np.vstack((X,X_in))
    Y = np.vstack((Y, np.repeat(i, np.shape(X_in)[0]).reshape(np.shape(X_in)[0],1)))
    '''
    data.append(g_obj.Scatter3d(x=X_in[:,0], y=X_in[:,1], z=X_in[:,2], mode='markers', \
                    marker=dict( size=3, color='rgb(0,0,255)', opacity=0.8)))
    data.append(g_obj.Scatter3d(x=X_out[:,0], y=X_out[:,1], z=X_out[:,2], mode='markers', \
                   marker=dict( size=3, color='rgb(255,0,0)', opacity=0.8)))
    
fig = g_obj.Figure(data=data)
fig['layout'].update(height=1200, width=1000)
py.plotly.iplot(fig)
    '''

#ZSCORE NORMALIZATION
X = stats.zscore(X, axis=0, ddof=1)


In [ ]:
'''
    MODEL SELECTION
'''
res = []
for hiddenL_n in [1,2,3]:
    for hiddenN_n in [7,10]:
        for learning_rate in [0.1,0.01]:
            for decay_ in [0.001, 0.0001]:
                
                sss = StratifiedShuffleSplit(n_splits=10, test_size=0.3)
                cvscores = []
                for train,test in sss.split(X,Y):
                    
                    he=tf.keras.initializers.he_normal()
                    model = tf.keras.models.Sequential()
                    
                    model.add(Dense(hiddenN_n, input_dim=3, activation='relu', kernel_initializer=he, bias_initializer=he))

                    for i in range(1,hiddenL_n): 
                        model.add(Dense(hiddenN_n, activation='relu', kernel_initializer=he, bias_initializer=he))

                    model.add(Dense(12, activation=tf.nn.softmax, kernel_initializer=he, bias_initializer=he))

                    sgd=tf.keras.optimizers.SGD(lr=learning_rate, momentum=0.9, decay=decay_)
                    model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy',\
                                 metrics=['accuracy'])
                    
                    model.fit(X[train], Y[train], epochs=10, verbose=2)
                    scores = model.evaluate(X[test], Y[test], verbose=0)
                    cvscores.append(scores[1])
                
                s = "hLayers: %d\thNeurons: %d\tlr: %.4f\tdecay: %.4f" % (hiddenL_n,hiddenN_n,learning_rate,decay_)
                res.append([s,np.mean(cvscores),np.std(cvscores,ddof=1)])
                

res.sort(key=lambda x: x[1], reverse=True)
print("SORTED RESULTS")        
for r in res:
    print(r[0])
    print("mean_acc: %.4f (std %.4f)\n" % (r[1], r[2]))